# Instal library

In [ ]:
# Impor library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Untuk visualisasi interaktif
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Atur gaya tampilan
plt.style.use('ggplot')


# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD, FastICA, NMF
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, SpectralEmbedding
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS, SpectralClustering, Birch
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.feature_selection import SelectKBest, mutual_info_regression, VarianceThreshold
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from scipy.stats import zscore, spearmanr, pearsonr
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
from statsmodels.stats.outliers_influence import variance_inflation_factor
from kneed import KneeLocator
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import library yang diperlukan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning & Preparation

In [43]:
# Baca file CSV
df = pd.read_csv('./data/employee_data.csv')

# Tampilkan 5 baris pertama untuk melihat struktur data
display(df.head())

,EmployeeId,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1,38,NaN,Travel_Frequently,1444,Human Resources,1,4,Other,1,...,2,80,1,7,2,3,6,2,1,2
1,2,37,1.0,Travel_Rarely,1141,Research & Development,11,2,Medical,1,...,1,80,0,15,2,1,1,0,0,0
2,3,51,1.0,Travel_Rarely,1323,Research & Development,4,4,Life Sciences,1,...,3,80,3,18,2,4,10,0,2,7
3,4,42,0.0,Travel_Frequently,555,Sales,26,3,Marketing,1,...,4,80,1,23,2,4,20,4,4,8
4,5,40,NaN,Travel_Rarely,1194,Research & Development,2,4,Medical,1,...,2,80,3,20,2,3,5,3,0,2


In [35]:
# Informasi tentang dataset
print("Ukuran dataset:", df.shape)
print("\nInformasi tentang tipe data:")
display(df.info())

Ukuran dataset: (1470, 35)

Informasi tentang tipe data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmployeeId                1470 non-null   int64  
 1   Age                       1470 non-null   int64  
 2   Attrition                 1058 non-null   float64
 3   BusinessTravel            1470 non-null   object 
 4   DailyRate                 1470 non-null   int64  
 5   Department                1470 non-null   object 
 6   DistanceFromHome          1470 non-null   int64  
 7   Education                 1470 non-null   int64  
 8   EducationField            1470 non-null   object 
 9   EmployeeCount             1470 non-null   int64  
 10  EnvironmentSatisfaction   1470 non-null   int64  
 11  Gender                    1470 non-null   object 
 12  HourlyRate                1470 non-null   int64  
 13  JobInv

None

In [36]:
# Statistik deskriptif data numerik
display(df.describe())

,EmployeeId,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1058.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,735.500000,36.923810,0.169187,802.485714,9.192517,2.912925,1.0,2.721769,65.891156,2.729932,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,424.496761,9.135373,0.375094,403.509100,8.106864,1.024165,0.0,1.093082,20.329428,0.711561,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,1.000000,18.000000,0.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,368.250000,30.000000,0.000000,465.000000,2.000000,2.000000,1.0,2.000000,48.000000,2.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,735.500000,36.000000,0.000000,802.000000,7.000000,3.000000,1.0,3.000000,66.000000,3.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,1102.750000,43.000000,0.000000,1157.000000,14.000000,4.000000,1.0,4.000000,83.750000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,1470.000000,60.000000,1.000000,1499.000000,29.000000,5.000000,1.0,4.000000,100.000000,4.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [44]:
df.isnull().sum()

EmployeeId                    0
Age                           0
Attrition                   412
BusinessTravel                0
DailyRate                     0
Department                    0
DistanceFromHome              0
Education                     0
EducationField                0
EmployeeCount                 0
EnvironmentSatisfaction       0
Gender                        0
HourlyRate                    0
JobInvolvement                0
JobLevel                      0
JobRole                       0
JobSatisfaction               0
MaritalStatus                 0
MonthlyIncome                 0
MonthlyRate                   0
NumCompaniesWorked            0
Over18                        0
OverTime                      0
PercentSalaryHike             0
PerformanceRating             0
RelationshipSatisfaction      0
StandardHours                 0
StockOptionLevel              0
TotalWorkingYears             0
TrainingTimesLastYear         0
WorkLifeBalance               0
YearsAtC

In [45]:
df.dropna(inplace=True)

In [46]:
df.isnull().sum()

EmployeeId                  0
Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSince

# Feature Engineering

## Salary Category

In [47]:
def salary_category(income):
    if income < 5000:
        return 'Rendah (< 5000)'
    elif income < 10000:
        return 'Sedang (5000-10000)'
    elif income < 15000:
        return 'Tinggi (10000-15000)'
    else:
        return 'Sangat Tinggi (>15000)'

df['SalaryCategory'] = df['MonthlyIncome'].apply(salary_category)

## Promotion Category

In [48]:
def promotion_category(years):
    if years == 0:
        return 'Baru Dipromosikan'
    elif years <= 2:
        return '1-2 Tahun'
    elif years <= 5:
        return '3-5 Tahun'
    else:
        return '> 5 Tahun'

df['PromotionCategory'] = df['YearsSinceLastPromotion'].apply(promotion_category)

## Age Category

In [49]:
# Buat kategori usia
def age_category(age):
    if age < 30:
        return '< 30'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    else:
        return '50+'

df['AgeGroup'] = df['Age'].apply(age_category)

## Satisfaction Category

In [50]:
satisfaction_cols = ['EnvironmentSatisfaction', 'JobSatisfaction', 
                     'RelationshipSatisfaction', 'WorkLifeBalance']
satisfaction_labels = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi', 4: 'Sangat Tinggi'}

df_satisfaction = df.copy()
for col in satisfaction_cols:
    df_satisfaction[col] = df_satisfaction[col].map(satisfaction_labels)

# Exploratory Data Analysis (EDA)

## Analisis Univariate

### Distribusi Attrition

In [83]:
attrition_counts = df['Attrition'].value_counts()
attrition_percentage = df['Attrition'].value_counts(normalize=True) * 100

attrition_df = pd.DataFrame({
    'Count': attrition_counts,
    'Percentage': attrition_percentage
})

print("Distribusi Attrition:")
display(attrition_df)

fig = px.pie(
    names=['Tidak Keluar', 'Keluar'] if 0 in attrition_counts.index else ['Keluar', 'Tidak Keluar'],
    values=attrition_counts.values,
    title='Persentase Karyawan yang Keluar vs Bertahan',
    color_discrete_sequence=px.colors.qualitative.Set3,
    hole=0.4
)
fig.update_traces(textinfo='percent+label', textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

fig.write_image("./visualisasi/presentase_attrition.jpg", scale=1)

Distribusi Attrition:


,Count,Percentage
Attrition,,
0.0,879,83.081285
1.0,179,16.918715


## Analisis Bivariate

### Attrition by Age

In [85]:
age_group_attrition = df.groupby('AgeGroup')['Attrition'].mean() * 100
age_group_counts = df.groupby('AgeGroup').size()

order = ['< 30', '30-39', '40-49', '50+']
age_group_attrition = age_group_attrition.reindex(order)
age_group_counts = age_group_counts.reindex(order)

age_group_df = pd.DataFrame({
    'Jumlah Karyawan': age_group_counts,
    'Tingkat Attrition (%)': age_group_attrition
})

display(age_group_df)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(
        x=age_group_df.index,
        y=age_group_df['Jumlah Karyawan'],
        name='Jumlah Karyawan',
        marker_color='royalblue',
        opacity=0.7
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=age_group_df.index,
        y=age_group_df['Tingkat Attrition (%)'],
        name='Tingkat Attrition (%)',
        marker_color='firebrick',
        mode='lines+markers',
        marker=dict(size=10),
        line=dict(width=3)
    ),
    secondary_y=True
)

fig.update_layout(
    title_text='Distribusi dan Tingkat Attrition Berdasarkan Kelompok Usia',
    template='plotly_white'
)

fig.update_xaxes(title_text='Kelompok Usia')
fig.update_yaxes(title_text='Jumlah Karyawan', secondary_y=False)
fig.update_yaxes(title_text='Tingkat Attrition (%)', secondary_y=True)

fig.show()
fig.write_image("./visualisasi/distribusi_usia_attrition.jpg", scale=1)

,Jumlah Karyawan,Tingkat Attrition (%)
AgeGroup,,
< 30,236,29.661017
30-39,437,15.102975
40-49,243,10.699588
50+,142,11.971831


### Attrition by Departemen

In [55]:
dept_attrition = df.groupby('Department')['Attrition'].mean().sort_values(ascending=False) * 100
dept_counts = df.groupby('Department').size()

dept_df = pd.DataFrame({
    'Jumlah Karyawan': dept_counts,
    'Tingkat Attrition (%)': dept_attrition
})

display(dept_df)

fig = px.bar(
    x=dept_attrition.index,
    y=dept_attrition.values,
    color=dept_attrition.values,
    text=dept_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Departemen',
    labels={'x': 'Departemen', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')

fig.show()

fig.write_image("./visualisasi/tingkat_attrition_departemen.jpg", scale=1)

,Jumlah Karyawan,Tingkat Attrition (%)
Department,,
Human Resources,38,15.789474
Research & Development,701,15.263909
Sales,319,20.689655


### Attrition by JobLevel

In [59]:
joblevel_attrition = df.groupby('JobLevel')['Attrition'].mean().sort_values(ascending=False) * 100
joblevel_counts = df.groupby('JobLevel').size()

joblevel_df = pd.DataFrame({
    'Tingkat Attrition (%)': joblevel_attrition,
    'Jumlah Karyawan': joblevel_counts
})

display(joblevel_df)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=joblevel_df.index,
    y=joblevel_df['Jumlah Karyawan'],
    name='Jumlah Karyawan',
    marker_color='lightblue',
    opacity=0.7,
    yaxis='y'
))

fig.add_trace(go.Scatter(
    x=joblevel_df.index,
    y=joblevel_df['Tingkat Attrition (%)'],
    name='Tingkat Attrition (%)',
    marker_color='red',
    mode='lines+markers',
    marker=dict(size=10),
    yaxis='y2'
))

fig.update_layout(
    title='Distribusi Attrition dan Jumlah Karyawan berdasarkan Job Level',
    xaxis=dict(title='Job Level'),
    yaxis=dict(title='Jumlah Karyawan', side='left', showgrid=False),
    yaxis2=dict(title='Tingkat Attrition (%)', side='right', overlaying='y', showgrid=False),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()

fig.write_image("./visualisasi/attrition_job_level.jpg", scale=2)

,Tingkat Attrition (%),Jumlah Karyawan
JobLevel,,
1,27.411168,394
2,10.164835,364
3,15.151515,165
4,5.000000,80
5,9.090909,55


### Attrition by MaritalStatus

In [64]:
marital_attrition = df.groupby('MaritalStatus')['Attrition'].mean() * 100
marital_counts = df.groupby('MaritalStatus').size()

marital_df = pd.DataFrame({
    'Jumlah Karyawan': marital_counts,
    'Tingkat Attrition (%)': marital_attrition
})

display(marital_df)

custom_labels = [f"{status}<br>Attrition: {attrition:.1f}%" 
                for status, attrition in zip(marital_attrition.index, marital_attrition.values)]

fig = go.Figure(go.Pie(
    labels=custom_labels,
    values=marital_counts.values,
    hole=0.4,
    textposition='outside',
    textinfo='percent', 
    pull=[0.05, 0.05, 0.05],
    marker=dict(
        colors=['#F9D876', '#F98B60', '#61D4D5'],
        line=dict(color='#FFFFFF', width=2)
    )
))

fig.update_layout(
    title='Distribusi Karyawan Berdasarkan Status Pernikahan',
    template='plotly_white',
    legend_title='Status Pernikahan & Tingkat Attrition',
    margin=dict(t=80, b=20, l=20, r=20),
    height=600,
    width=800
)

fig.add_annotation(
    text="Status<br>Pernikahan",
    x=0.5, y=0.5,
    font_size=14,
    showarrow=False
)

fig.show()

fig.write_image("./visualisasi/distribusi_status_pernikahan.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
MaritalStatus,,
Divorced,242,9.504132
Married,464,13.362069
Single,352,26.704545


### Attrition by Distance

In [68]:
distance_stats = df.groupby('Attrition')['DistanceFromHome'].agg(['mean', 'median', 'min', 'max', 'std']).reset_index()

distance_stats['Attrition'] = distance_stats['Attrition'].map({0: 'Tidak Keluar', 1: 'Keluar'})

display(distance_stats)

fig = px.box(
    df, 
    x='Attrition', 
    y='DistanceFromHome',
    color='Attrition',
    title='Hubungan Jarak dari Rumah dengan Attrition',
    labels={'DistanceFromHome': 'Jarak dari Rumah (km)', 'Attrition': 'Status Keluar'},
    color_discrete_sequence=['#636EFA', '#EF553B']
)
fig.update_layout(template='plotly_white')

fig.show()

fig.write_image("./visualisasi/jarak_rumah_attrition.jpg", scale=2)

,Attrition,mean,median,min,max,std
0,Tidak Keluar,8.695108,7.0,1,29,7.909964
1,Keluar,10.368715,8.0,1,29,8.541263


### Attrtion by Satisfaction

In [72]:
df_satisfaction = df.copy()
for col in satisfaction_cols:
    df_satisfaction[col] = df_satisfaction[col].map(satisfaction_labels)

display(df_satisfaction)

fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=['Kepuasan Lingkungan', 'Kepuasan Kerja', 
                                  'Kepuasan Hubungan', 'Work Life Balance'],)

col_positions = [
    ('EnvironmentSatisfaction', 1, 1),
    ('JobSatisfaction', 1, 2),
    ('RelationshipSatisfaction', 2, 1),
    ('WorkLifeBalance', 2, 2)
]

for col, row, col_pos in col_positions:
    attrition_by_satisfaction = df_satisfaction.groupby(col)['Attrition'].mean() * 100
    
    fig.add_trace(
        go.Bar(
            x=attrition_by_satisfaction.index,
            y=attrition_by_satisfaction.values,
            text=[f'{val:.1f}%' for val in attrition_by_satisfaction.values],
            textposition='auto',
            marker_color='lightseagreen'
        ),
        row=row, col=col_pos
    )

fig.update_layout(
    height=600,
    width=900,
    showlegend=False,
    title_text='Tingkat Attrition (%) Berdasarkan Faktor Kepuasan',
    template='plotly_white'
)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=12, color='#000000')

fig.update_xaxes(title_text='Tingkat Kepuasan')
fig.update_yaxes(title_text='Tingkat Attrition (%)')

fig.show()

fig.write_image("./visualisasi/tingkat_attrition_kepuasan.jpg", scale=2)

,EmployeeId,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,SalaryCategory,PromotionCategory,AgeGroup
1,2,37,1.0,Travel_Rarely,1141,Research & Development,11,2,Medical,1,...,15,2,Rendah,1,0,0,0,Rendah (< 5000),Baru Dipromosikan,30-39
2,3,51,1.0,Travel_Rarely,1323,Research & Development,4,4,Life Sciences,1,...,18,2,Sangat Tinggi,10,0,2,7,Rendah (< 5000),1-2 Tahun,50+
3,4,42,0.0,Travel_Frequently,555,Sales,26,3,Marketing,1,...,23,2,Sangat Tinggi,20,4,4,8,Tinggi (10000-15000),3-5 Tahun,40-49
6,7,40,0.0,Travel_Rarely,1124,Sales,1,2,Medical,1,...,6,2,Sedang,4,3,0,2,Sedang (5000-10000),Baru Dipromosikan,40-49
7,8,55,1.0,Travel_Rarely,725,Research & Development,2,3,Medical,1,...,24,2,Tinggi,5,2,1,4,Sangat Tinggi (>15000),1-2 Tahun,50+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,1465,28,1.0,Non-Travel,1366,Research & Development,24,2,Technical Degree,1,...,10,2,Sedang,10,7,1,9,Sedang (5000-10000),1-2 Tahun,< 30
1465,1466,38,0.0,Travel_Rarely,168,Research & Development,1,3,Life Sciences,1,...,10,4,Sangat Tinggi,1,0,0,0,Sedang (5000-10000),Baru Dipromosikan,30-39
1467,1468,28,1.0,Travel_Rarely,1485,Research & Development,12,1,Life Sciences,1,...,1,4,Sedang,1,1,0,0,Rendah (< 5000),Baru Dipromosikan,< 30
1468,1469,40,0.0,Non-Travel,458,Research & Development,16,2,Life Sciences,1,...,6,0,Tinggi,4,2,0,0,Rendah (< 5000),Baru Dipromosikan,40-49


### Attrition by JobRole

In [73]:
jobrole_attrition = df.groupby('JobRole')['Attrition'].mean().sort_values(ascending=False) * 100
jobrole_counts = df.groupby('JobRole').size()

jobrole_df = pd.DataFrame({
    'Tingkat Attrition (%)': jobrole_attrition,
    'Jumlah Karyawan': jobrole_counts
})

display(jobrole_df)

fig = px.bar(
    x=jobrole_attrition.index,
    y=jobrole_attrition.values,
    color=jobrole_attrition.values,
    text=jobrole_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Job Role',
    labels={'x': 'Job Role', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(
    coloraxis_showscale=False,
    xaxis_tickangle=-45,
    template='plotly_white'
)
fig.show()

fig.write_image("./visualisasi/tingkat_attrition_job_role.jpg", scale=2)

,Tingkat Attrition (%),Jumlah Karyawan
JobRole,,
Healthcare Representative,9.090909,88
Human Resources,20.000000,30
Laboratory Technician,26.063830,188
Manager,6.329114,79
Manufacturing Director,6.542056,107
Research Director,3.225806,62
Research Scientist,17.757009,214
Sales Executive,16.810345,232
Sales Representative,43.103448,58


### Attrition by Overtime

In [74]:
overtime_attrition = df.groupby('OverTime')['Attrition'].mean() * 100
overtime_counts = df.groupby('OverTime').size()

overtime_df = pd.DataFrame({
    'Jumlah Karyawan': overtime_counts,
    'Tingkat Attrition (%)': overtime_attrition
})

display(overtime_df)

fig = px.bar(
    x=overtime_attrition.index,
    y=overtime_attrition.values,
    color=overtime_attrition.values,
    text=overtime_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Status Overtime',
    labels={'x': 'Overtime', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Reds'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()

fig.write_image("./visualisasi/tingkat_attrition_overtime.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
OverTime,,
No,751,10.785619
Yes,307,31.921824


### Attrition by Salary Category

In [75]:
salary_attrition = df.groupby('SalaryCategory')['Attrition'].mean() * 100
salary_counts = df.groupby('SalaryCategory').size()

order = ['Rendah (< 5000)', 'Sedang (5000-10000)', 'Tinggi (10000-15000)', 'Sangat Tinggi (>15000)']
salary_attrition = salary_attrition.reindex(order)
salary_counts = salary_counts.reindex(order)

salary_df = pd.DataFrame({
    'Jumlah Karyawan': salary_counts,
    'Tingkat Attrition (%)': salary_attrition
})

display(salary_df)

fig = px.bar(
    x=salary_attrition.index,
    y=salary_attrition.values,
    color=salary_attrition.values,
    text=salary_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Kategori Gaji',
    labels={'x': 'Kategori Gaji', 'y': 'Tingkat Attrition (%)'},
    color_continuous_scale='Greens_r'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_kategori_gaji.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
SalaryCategory,,
Rendah (< 5000),542,22.878229
Sedang (5000-10000),297,11.784512
Tinggi (10000-15000),117,12.820513
Sangat Tinggi (>15000),102,4.901961


### Attrition by Gender

In [76]:
gender_attrition = df.groupby('Gender')['Attrition'].mean() * 100
gender_counts = df.groupby('Gender').size()

gender_df = pd.DataFrame({
    'Jumlah Karyawan': gender_counts,
    'Tingkat Attrition (%)': gender_attrition
})

display(gender_df)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=gender_counts.index,
    y=gender_counts.values,
    name='Jumlah Karyawan',
    marker_color='skyblue',
    opacity=0.7,
    text=gender_counts.values,
    textposition='auto',
    yaxis='y'
))

fig.add_trace(go.Scatter(
    x=gender_attrition.index,
    y=gender_attrition.values,
    name='Tingkat Attrition (%)',
    marker_color='red',
    mode='lines+markers+text',
    marker=dict(size=12),
    text=[f'{val:.1f}%' for val in gender_attrition.values],
    textposition='top center',
    yaxis='y2'
))

fig.update_layout(
    title='Jumlah Karyawan dan Tingkat Attrition Berdasarkan Jenis Kelamin',
    xaxis=dict(title='Jenis Kelamin'),
    yaxis=dict(title='Jumlah Karyawan', side='left', showgrid=False),
    yaxis2=dict(title='Tingkat Attrition (%)', side='right', overlaying='y', showgrid=False),
    legend=dict(x=0.01, y=0.99),
    template='plotly_white'
)

fig.show()
fig.write_image("./visualisasi/tingkat_attrition_jenis_kelamin.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
Gender,,
Female,438,16.210046
Male,620,17.419355


### Attrition by Education Background

In [77]:
education_field_attrition = df.groupby('EducationField')['Attrition'].mean() * 100
education_field_counts = df.groupby('EducationField').size()

education_field_df = pd.DataFrame({
    'Jumlah Karyawan': education_field_counts,
    'Tingkat Attrition (%)': education_field_attrition
})

education_field_df = education_field_df.sort_values('Tingkat Attrition (%)', ascending=False)
display(education_field_df)

fig = px.bar(
    education_field_df,
    x=education_field_df.index,
    y='Tingkat Attrition (%)',
    color='Tingkat Attrition (%)',
    text=education_field_df['Tingkat Attrition (%)'].round(1),
    title='Tingkat Attrition (%) Berdasarkan Bidang Pendidikan',
    labels={'y': 'Tingkat Attrition (%)', 'x': 'Bidang Pendidikan'},
    color_continuous_scale='Blues_r'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(coloraxis_showscale=False, template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_bidang_pendidikan.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
EducationField,,
Technical Degree,96,26.041667
Marketing,122,21.311475
Other,59,16.949153
Life Sciences,436,16.055046
Medical,330,13.939394
Human Resources,15,13.333333


### Attrition by LastTimePromoted

In [79]:
promotion_attrition = df.groupby('PromotionCategory')['Attrition'].mean() * 100
promotion_counts = df.groupby('PromotionCategory').size()

order = ['Baru Dipromosikan', '1-2 Tahun', '3-5 Tahun', '> 5 Tahun']
promotion_attrition = promotion_attrition.reindex(order)
promotion_counts = promotion_counts.reindex(order)

promotion_df = pd.DataFrame({
    'Jumlah Karyawan': promotion_counts,
    'Tingkat Attrition (%)': promotion_attrition
})

display(promotion_df)

fig = px.line(
    x=promotion_attrition.index,
    y=promotion_attrition.values,
    markers=True,
    text=promotion_attrition.values.round(1),
    title='Tingkat Attrition (%) Berdasarkan Tahun Sejak Promosi Terakhir',
    labels={'x': 'Tahun Sejak Promosi Terakhir', 'y': 'Tingkat Attrition (%)'},
)
fig.update_traces(
    marker=dict(size=10, color='blue'),
    line=dict(width=3, color='purple'),
    textposition='top center'
)
fig.update_layout(template='plotly_white')
fig.show()
fig.write_image("./visualisasi/tingkat_attrition_promotion.jpg", scale=2)

,Jumlah Karyawan,Tingkat Attrition (%)
PromotionCategory,,
Baru Dipromosikan,427,19.437939
1-2 Tahun,358,15.642458
3-5 Tahun,118,11.864407
> 5 Tahun,155,16.774194


,Jumlah Karyawan,Tingkat Attrition (%)
AgeGroup,,
< 30,236,29.661017
30-39,437,15.102975
40-49,243,10.699588
50+,142,11.971831


## Multivariate

In [86]:
# 1. Analisis multivariate: Overtime, Job Satisfaction, dan Attrition
# Membuat tabel pivot untuk melihat interaksi antara overtime dan tingkat kepuasan kerja
pivot_overtime_satisfaction = pd.crosstab(
    index=df['OverTime'],
    columns=df['JobSatisfaction'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Ubah nilai job satisfaction menjadi kategori untuk memudahkan pembacaan
satisfaction_labels = {1: 'Rendah', 2: 'Sedang', 3: 'Tinggi', 4: 'Sangat Tinggi'}
pivot_overtime_satisfaction.columns = [satisfaction_labels[col] for col in pivot_overtime_satisfaction.columns]

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Overtime dan Job Satisfaction:")
display(pivot_overtime_satisfaction)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_overtime_satisfaction,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='RdYlGn_r',
    title='Tingkat Attrition (%) Berdasarkan Overtime dan Job Satisfaction',
    labels=dict(x='Tingkat Kepuasan Kerja', y='Overtime', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_overtime_jobsatisfaction.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_overtime_jobsatisfaction.jpg'")

Tingkat Attrition (%) berdasarkan Overtime dan Job Satisfaction:


,Rendah,Sedang,Tinggi,Sangat Tinggi
OverTime,,,,
No,18.309859,8.510638,11.403509,7.083333
Yes,31.746032,37.037037,40.000000,22.000000


Visualisasi telah disimpan sebagai './visualisasi/attrition_overtime_jobsatisfaction.jpg'


In [87]:
# 2. Analisis multivariate: Department, Job Level, dan Attrition
# Membuat tabel pivot untuk melihat interaksi antara department dan job level
pivot_dept_joblevel = pd.crosstab(
    index=df['Department'],
    columns=df['JobLevel'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Department dan Job Level:")
display(pivot_dept_joblevel)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_dept_joblevel,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='Viridis_r',
    title='Tingkat Attrition (%) Berdasarkan Department dan Job Level',
    labels=dict(x='Job Level', y='Department', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_department_joblevel.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_department_joblevel.jpg'")

Tingkat Attrition (%) berdasarkan Department dan Job Level:


JobLevel,1,2,3,4,5
Department,,,,,
Human Resources,22.222222,0.000000,50.000000,0.000000,0.0
Research & Development,24.844720,5.851064,12.121212,1.923077,7.5
Sales,44.444444,15.476190,17.741935,12.000000,20.0


Visualisasi telah disimpan sebagai './visualisasi/attrition_department_joblevel.jpg'


In [88]:
# 3. Analisis multivariate: Gaji, Jarak dari Rumah, Overtime, dan Attrition
# Scatter plot dengan multiple dimensions
fig = px.scatter(
    df,
    x='MonthlyIncome',
    y='DistanceFromHome',
    color='Attrition',
    facet_col='OverTime',
    opacity=0.7,
    hover_name='JobRole',
    hover_data=['Age', 'YearsAtCompany', 'JobSatisfaction'],
    title='Hubungan antara Gaji, Jarak dari Rumah, Overtime dengan Attrition',
    labels={
        'MonthlyIncome': 'Gaji Bulanan',
        'DistanceFromHome': 'Jarak dari Rumah (km)',
        'Attrition': 'Status Attrition',
        'OverTime': 'Status Overtime'
    },
    color_discrete_sequence=['green', 'red']
)
fig.update_layout(
    template='plotly_white',
    height=600
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_income_distance_overtime.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_income_distance_overtime.jpg'")

Visualisasi telah disimpan sebagai './visualisasi/attrition_income_distance_overtime.jpg'


In [89]:
# 4. Analisis multivariate: Usia, Job Level, dan Attrition
# Buat kategori usia
if 'AgeGroup' not in df.columns:
    df['AgeGroup'] = pd.cut(
        df['Age'],
        bins=[20, 30, 40, 50, 60],
        labels=['20-29', '30-39', '40-49', '50-59']
    )

# Buat tabel pivot
pivot_age_joblevel = pd.crosstab(
    index=df['AgeGroup'],
    columns=df['JobLevel'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Tampilkan dataframe
print("Tingkat Attrition (%) berdasarkan Kelompok Usia dan Job Level:")
display(pivot_age_joblevel)

# Visualisasi dengan heatmap
fig = px.imshow(
    pivot_age_joblevel,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='Plasma_r',
    title='Tingkat Attrition (%) Berdasarkan Kelompok Usia dan Job Level',
    labels=dict(x='Job Level', y='Kelompok Usia', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_age_joblevel.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_age_joblevel.jpg'")

Tingkat Attrition (%) berdasarkan Kelompok Usia dan Job Level:


JobLevel,1,2,3,4,5
AgeGroup,,,,,
30-39,22.560976,10.439560,11.111111,12.500000,0.000000
40-49,14.285714,7.777778,18.181818,6.250000,7.142857
50+,21.739130,14.285714,14.814815,2.564103,12.000000
< 30,37.341772,10.937500,30.769231,0.000000,NaN


Visualisasi telah disimpan sebagai './visualisasi/attrition_age_joblevel.jpg'


In [66]:
# Analisis kombinasi overtime dan work-life balance
cross_analysis = pd.crosstab(
    index=df['OverTime'],
    columns=df['WorkLifeBalance'],
    values=df['Attrition'],
    aggfunc='mean'
) * 100

# Ubah nilai worklife balance menjadi label
cross_analysis.columns = ['Buruk', 'Sedang', 'Baik', 'Sangat Baik']

display(cross_analysis)

# Visualisasi dengan heatmap
fig = px.imshow(
    cross_analysis,
    text_auto='.1f',
    aspect='auto',
    color_continuous_scale='RdYlGn_r',
    title='Tingkat Attrition (%) Berdasarkan Overtime dan Work-Life Balance',
    labels=dict(x='Work-Life Balance', y='Overtime', color='Attrition Rate (%)')
)
fig.update_layout(template='plotly_white')
fig.show()

,Buruk,Sedang,Baik,Sangat Baik
OverTime,,,,
No,25.0,11.627907,8.333333,16.091954
Yes,45.0,31.645570,30.769231,30.769231


In [90]:
# 5. Analisis multivariate: 3D Scatter Plot - Gaji, Tahun di Perusahaan, Jarak dari Rumah, dan Attrition
fig = px.scatter_3d(
    df,
    x='MonthlyIncome',
    y='YearsAtCompany',
    z='DistanceFromHome',
    color='Attrition',
    opacity=0.7,
    size='JobLevel',
    hover_name='JobRole',
    title='Hubungan 3D antara Gaji, Tahun Bekerja, Jarak dari Rumah, dan Attrition',
    labels={
        'MonthlyIncome': 'Gaji Bulanan',
        'YearsAtCompany': 'Tahun di Perusahaan',
        'DistanceFromHome': 'Jarak dari Rumah (km)',
        'Attrition': 'Status Attrition',
        'JobLevel': 'Level Pekerjaan'
    },
    color_discrete_sequence=['green', 'red']
)
fig.update_layout(
    template='plotly_white',
    scene=dict(
        xaxis_title='Gaji Bulanan',
        yaxis_title='Tahun di Perusahaan',
        zaxis_title='Jarak dari Rumah (km)'
    )
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_3d_income_years_distance.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_3d_income_years_distance.jpg'")

Visualisasi telah disimpan sebagai './visualisasi/attrition_3d_income_years_distance.jpg'


In [91]:
# 6. Analisis multivariate: Parallel Coordinates Plot untuk melihat banyak dimensi sekaligus
# Pilih kolom numerik yang relevan
cols_for_parallel = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'YearsAtCompany', 
                     'WorkLifeBalance', 'JobSatisfaction', 'JobLevel']

# Buat Parallel Coordinates Plot
fig = px.parallel_coordinates(
    df, 
    color='Attrition',
    dimensions=cols_for_parallel,
    color_continuous_scale=px.colors.diverging.Tealrose,
    title='Parallel Coordinates Plot untuk Berbagai Dimensi dan Attrition'
)
fig.update_layout(
    template='plotly_white',
    height=600
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_parallel_coordinates.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_parallel_coordinates.jpg'")

Visualisasi telah disimpan sebagai './visualisasi/attrition_parallel_coordinates.jpg'


In [92]:
# 7. Analisis multivariate: Bubble Chart untuk Department, Job Role, Job Satisfaction dan Attrition
# Hitung tingkat attrition berdasarkan kombinasi department dan job role
attrition_by_dept_role = df.groupby(['Department', 'JobRole'])['Attrition'].mean() * 100
count_by_dept_role = df.groupby(['Department', 'JobRole']).size()
satisfaction_by_dept_role = df.groupby(['Department', 'JobRole'])['JobSatisfaction'].mean()

# Buat dataframe
multivariate_df = pd.DataFrame({
    'Tingkat Attrition (%)': attrition_by_dept_role,
    'Jumlah Karyawan': count_by_dept_role,
    'Rata-rata Kepuasan Kerja': satisfaction_by_dept_role
}).reset_index()

# Tampilkan dataframe
print("Tingkat Attrition (%), Jumlah Karyawan, dan Rata-rata Kepuasan Kerja berdasarkan Department dan Job Role:")
display(multivariate_df.sort_values('Tingkat Attrition (%)', ascending=False))

# Visualisasi dengan bubble chart
fig = px.scatter(
    multivariate_df,
    x='Rata-rata Kepuasan Kerja',
    y='Tingkat Attrition (%)',
    size='Jumlah Karyawan',
    color='Department',
    hover_name='JobRole',
    text='JobRole',
    title='Tingkat Attrition (%) vs Kepuasan Kerja berdasarkan Department dan Job Role',
    labels={
        'Rata-rata Kepuasan Kerja': 'Rata-rata Kepuasan Kerja (1-4)',
        'Tingkat Attrition (%)': 'Tingkat Attrition (%)'
    }
)
fig.update_traces(
    textposition='top center',
    marker=dict(sizemode='area', sizeref=0.1)
)
fig.update_layout(
    template='plotly_white',
    height=800,
    width=1000
)
fig.show()

# Simpan gambar
fig.write_image("./visualisasi/attrition_dept_role_satisfaction.jpg", scale=2)
print("Visualisasi telah disimpan sebagai './visualisasi/attrition_dept_role_satisfaction.jpg'")

Tingkat Attrition (%), Jumlah Karyawan, dan Rata-rata Kepuasan Kerja berdasarkan Department dan Job Role:


,Department,JobRole,Tingkat Attrition (%),Jumlah Karyawan,Rata-rata Kepuasan Kerja
10,Sales,Sales Representative,43.103448,58,2.758621
3,Research & Development,Laboratory Technician,26.063830,188,2.813830
0,Human Resources,Human Resources,20.000000,30,2.366667
7,Research & Development,Research Scientist,17.757009,214,2.780374
9,Sales,Sales Executive,16.810345,232,2.754310
2,Research & Development,Healthcare Representative,9.090909,88,2.829545
4,Research & Development,Manager,7.142857,42,2.452381
8,Sales,Manager,6.896552,29,2.827586
5,Research & Development,Manufacturing Director,6.542056,107,2.710280
6,Research & Development,Research Director,3.225806,62,2.725806


Visualisasi telah disimpan sebagai './visualisasi/attrition_dept_role_satisfaction.jpg'


In [44]:
# Pastikan semua feature engineering dan variabel baru yang ingin disimpan sudah dibuat

# Buat daftar variabel baru yang telah dibuat
new_variables = [
    'SalaryCategory',        # Kategori gaji
    'PromotionCategory',     # Kategori tahun sejak promosi terakhir
    'AgeGroup',              # Kelompok usia
    'DistanceCategory'       # Kategori jarak dari rumah
]

# Periksa variabel mana yang sudah ada di dataframe
existing_variables = [var for var in new_variables if var in df.columns]

# Jika ada variabel yang belum dibuat, buat terlebih dahulu
if 'SalaryCategory' not in existing_variables:
    df['SalaryCategory'] = df['MonthlyIncome'].apply(lambda income: 
        'Rendah (< 5000)' if income < 5000 else
        'Sedang (5000-10000)' if income < 10000 else
        'Tinggi (10000-15000)' if income < 15000 else
        'Sangat Tinggi (>15000)'
    )

if 'PromotionCategory' not in existing_variables:
    df['PromotionCategory'] = df['YearsSinceLastPromotion'].apply(lambda years:
        'Baru Dipromosikan' if years == 0 else
        '1-2 Tahun' if years <= 2 else
        '3-5 Tahun' if years <= 5 else
        '> 5 Tahun'
    )

if 'AgeGroup' not in existing_variables:
    df['AgeGroup'] = df['Age'].apply(lambda age:
        '< 30' if age < 30 else
        '30-39' if age < 40 else
        '40-49' if age < 50 else
        '50+'
    )

if 'DistanceCategory' not in existing_variables:
    df['DistanceCategory'] = pd.cut(
        df['DistanceFromHome'],
        bins=[0, 5, 10, 20, 30],
        labels=['0-5 km', '6-10 km', '11-20 km', '21-30 km']
    )

# Tampilkan kolom-kolom yang ada di data final
print("Kolom-kolom dalam dataset final:")
display(pd.DataFrame({'Nama Kolom': df.columns}))

# Simpan data final
df.to_csv('./data_final_employee.csv', index=False)
print("Data final telah disimpan sebagai 'data_final_employee.csv'")

# Jika ingin menyimpan dalam format Excel
try:
    df.to_excel('./data_final_employee.xlsx', index=False)
    print("Data final juga telah disimpan sebagai 'data_final_employee.xlsx'")
except Exception as e:
    print(f"Gagal menyimpan dalam format Excel: {e}")
    print("Tip: Jika ingin menyimpan dalam format Excel, pastikan library openpyxl sudah terinstal.")
    print("Anda dapat menginstalnya dengan 'pip install openpyxl'")

Kolom-kolom dalam dataset final:


,Nama Kolom
0,EmployeeId
1,Age
2,Attrition
3,BusinessTravel
4,DailyRate
5,Department
6,DistanceFromHome
7,Education
8,EducationField
9,EmployeeCount


Data final telah disimpan sebagai 'data_final_employee.csv'
Gagal menyimpan dalam format Excel: No module named 'openpyxl'
Tip: Jika ingin menyimpan dalam format Excel, pastikan library openpyxl sudah terinstal.
Anda dapat menginstalnya dengan 'pip install openpyxl'


# Segmentation Using Unsupervised Learning

In [ ]:
df = pd.read_csv('data_final_employee.csv')
print(f"Ukuran data awal: {df.shape}")

In [ ]:
# 1. Feature Engineering yang lebih canggih
print("\n=== Feature Engineering ===")
df_eng = df.copy()

# Konversi kolom kategoris ke numerik
df_eng['OverTime_Numeric'] = df_eng['OverTime'].map({'Yes': 1, 'No': 0})

# Transformasi berdasarkan domain knowledge
df_eng['Income_Level'] = df_eng['MonthlyIncome'] / df_eng['JobLevel']
df_eng['Satisfaction_Index'] = (df_eng['JobSatisfaction'] + df_eng['EnvironmentSatisfaction'] + 
                               df_eng['WorkLifeBalance'] + df_eng['RelationshipSatisfaction']) / 4
df_eng['Career_Progress'] = df_eng['JobLevel'] / (df_eng['YearsSinceLastPromotion'] + 1)
df_eng['Stability_Factor'] = df_eng['YearsAtCompany'] / (df_eng['NumCompaniesWorked'] + 1)
df_eng['Work_Life_Balance_Impact'] = df_eng['WorkLifeBalance'] * (1 - df_eng['OverTime_Numeric'])
df_eng['Distance_Strain'] = df_eng['DistanceFromHome'] / (df_eng['WorkLifeBalance'] + 0.1)
df_eng['Promotion_Expectation'] = df_eng['JobLevel'] / (df_eng['TotalWorkingYears'] + 1)
df_eng['Experience_vs_Tenure'] = (df_eng['TotalWorkingYears'] - df_eng['YearsAtCompany']) / (df_eng['TotalWorkingYears'] + 1)
df_eng['Loyalty_Index'] = (df_eng['YearsAtCompany'] * df_eng['JobSatisfaction']) / (df_eng['NumCompaniesWorked'] + 1)

In [ ]:
# 2. Pemilihan fitur penting secara manual berdasarkan domain knowledge
# Karena RandomForest error dengan NaN, kita gunakan pendekatan manual
important_features = [
    'MonthlyIncome', 'OverTime_Numeric', 'JobSatisfaction', 
    'DistanceFromHome', 'YearsAtCompany', 'Age',
    'JobLevel', 'TotalWorkingYears', 'YearsSinceLastPromotion',
    'WorkLifeBalance', 'EnvironmentSatisfaction', 'Satisfaction_Index',
    'Career_Progress', 'Distance_Strain', 'Loyalty_Index',
    'Work_Life_Balance_Impact', 'Stability_Factor', 'Income_Level',
    'Promotion_Expectation', 'Experience_vs_Tenure'
]

In [ ]:
# 3. Seleksi fitur untuk clustering
print("\n=== Persiapan Data untuk Clustering ===")
X_features = df_eng[important_features].copy()

# Tangani missing values jika ada
X_features = X_features.fillna(X_features.median())

In [ ]:
# 4. Normalisasi/transformasi non-linear
print("\n=== Transformasi Data ===")
# Kita akan mencoba beberapa metode transformasi dan memilih yang terbaik
scalers = {
    'RobustScaler': RobustScaler(),
    'PowerTransformer': PowerTransformer(method='yeo-johnson')
}

X_transformed = {}
for name, scaler in scalers.items():
    X_transformed[name] = scaler.fit_transform(X_features)

In [ ]:
# 5. Reduksi dimensi canggih
print("\n=== Reduksi Dimensi ===")
# Untuk visualisasi 2D, kita hanya perlu 2 komponen
reduction_methods = {
    'PCA': PCA(n_components=2, random_state=42),
    'KernelPCA_rbf': KernelPCA(n_components=2, kernel='rbf', random_state=42),
    'KernelPCA_poly': KernelPCA(n_components=2, kernel='poly', random_state=42),
    'TruncatedSVD': TruncatedSVD(n_components=2, random_state=42)
}

X_reduced = {}
for transform_name, X_trans in X_transformed.items():
    for method_name, reducer in reduction_methods.items():
        try:
            X_reduced[f"{transform_name}_{method_name}"] = reducer.fit_transform(X_trans)
            # Print explained variance if available
            if hasattr(reducer, 'explained_variance_ratio_'):
                print(f"{transform_name}_{method_name} explained variance: {np.sum(reducer.explained_variance_ratio_):.4f}")
        except Exception as e:
            print(f"Error with {transform_name}_{method_name}: {e}")

In [ ]:
# 6. Clustering canggih
print("\n=== Clustering ===")
# Kita akan mencoba berbagai algoritma clustering
clustering_methods = {
    'KMeans': KMeans(n_clusters=3, random_state=42, n_init=50, max_iter=1000),
    'AgglomerativeClustering': AgglomerativeClustering(n_clusters=3),
    'GaussianMixture': GaussianMixture(n_components=3, random_state=42, n_init=10, max_iter=1000, 
                                      covariance_type='full', reg_covar=1e-5),
    'SpectralClustering': SpectralClustering(n_clusters=3, random_state=42, assign_labels='kmeans', 
                                           n_init=50, affinity='nearest_neighbors')
}

# Tabel untuk menyimpan skor silhouette
scores = []

# Coba semua kombinasi dan cari yang terbaik
for data_name, X_data in X_reduced.items():
    for cluster_name, clusterer in clustering_methods.items():
        try:
            # Fit clusterer
            if cluster_name == 'GaussianMixture':
                clusterer.fit(X_data)
                labels = clusterer.predict(X_data)
            else:
                labels = clusterer.fit_predict(X_data)
            
            # Calculate scores
            silhouette = silhouette_score(X_data, labels)
            ch_score = calinski_harabasz_score(X_data, labels)
            db_score = davies_bouldin_score(X_data, labels)
            
            # Save scores
            scores.append({
                'Data': data_name,
                'Clustering': cluster_name,
                'Silhouette': silhouette,
                'Calinski-Harabasz': ch_score,
                'Davies-Bouldin': db_score
            })
            
            print(f"{data_name} + {cluster_name}: Silhouette={silhouette:.4f}, CH={ch_score:.1f}, DB={db_score:.4f}")
        except Exception as e:
            print(f"Error with {data_name} + {cluster_name}: {e}")

# Convert to DataFrame and sort by silhouette score
scores_df = pd.DataFrame(scores).sort_values('Silhouette', ascending=False)
print("\nTop 5 combinations:")
print(scores_df.head())

# Get the best combination
best_combination = scores_df.iloc[0]
best_data_name = best_combination['Data']
best_cluster_name = best_combination['Clustering']
best_silhouette = best_combination['Silhouette']

print(f"\nBest combination: {best_data_name} + {best_cluster_name}")
print(f"Silhouette score: {best_silhouette:.4f}")

In [ ]:
# 7. Menghasilkan cluster terbaik
# Get the data and clusterer from the best combination
best_X = X_reduced[best_data_name]
best_clusterer = clustering_methods[best_cluster_name]

# Fit the best clusterer
if best_cluster_name == 'GaussianMixture':
    best_clusterer.fit(best_X)
    cluster_labels = best_clusterer.predict(best_X)
else:
    cluster_labels = best_clusterer.fit_predict(best_X)

# Add cluster labels to dataframe
df_eng['Cluster'] = cluster_labels

In [ ]:
# 8. Interpretasi cluster berdasarkan attrition
# Pastikan attrition adalah numerik (0 atau 1)
if df_eng['Attrition'].dtype == 'object':
    df_eng['Attrition'] = df_eng['Attrition'].astype(float)

# Tangani missing values dalam attrition (jika ada)
if df_eng['Attrition'].isna().any():
    print(f"Ditemukan {df_eng['Attrition'].isna().sum()} nilai hilang dalam Attrition")
    # Impute dengan modus
    mode_attrition = df_eng['Attrition'].mode()[0]
    df_eng['Attrition'] = df_eng['Attrition'].fillna(mode_attrition)

# Calculate attrition rate by cluster
attrition_by_cluster = df_eng.groupby('Cluster')['Attrition'].mean() * 100
cluster_sizes = df_eng.groupby('Cluster').size()

# Sort clusters by attrition rate
sorted_clusters = attrition_by_cluster.sort_values().index

# Map cluster IDs to risk labels
cluster_map = {
    sorted_clusters[0]: 'Risiko Rendah',
    sorted_clusters[1]: 'Risiko Sedang', 
    sorted_clusters[2]: 'Risiko Tinggi'
}

# Add cluster labels to dataframe
df_eng['Cluster_Label'] = df_eng['Cluster'].map(cluster_map)

# Display attrition by cluster
print("\nAttrition rate by cluster:")
for cluster, rate in attrition_by_cluster.items():
    label = cluster_map[cluster]
    size = cluster_sizes[cluster]
    print(f"{label} (Cluster {cluster}): {rate:.2f}% ({size} employees)")

In [ ]:
# 9. Visualisasi 2D dengan skema warna colorblind-friendly
# Add the best reduced dimensions to the dataframe
df_eng['Dim1'] = best_X[:, 0]
df_eng['Dim2'] = best_X[:, 1]

# Create the 2D scatter plot with colorblind-friendly palette
color_map = {
    'Risiko Rendah': '#0072B2',    # Biru tua
    'Risiko Sedang': '#E69F00',    # Oranye
    'Risiko Tinggi': '#CC79A7'     # Pink/Magenta
}

fig = px.scatter(
    df_eng, 
    x='Dim1', 
    y='Dim2',
    color='Cluster_Label',
    symbol='Attrition',
    size='MonthlyIncome',
    hover_name='EmployeeId',
    hover_data=[
        'Age', 'JobSatisfaction', 'OverTime', 'MonthlyIncome', 
        'YearsAtCompany', 'DistanceFromHome', 'Satisfaction_Index'
    ],
    title=f'Segmentasi Karyawan - Silhouette Score: {best_silhouette:.4f}',
    color_discrete_map=color_map,
    symbol_map={0: 'circle', 1: 'diamond'},
    labels={
        'Dim1': 'Dimension 1',
        'Dim2': 'Dimension 2',
        'Cluster_Label': 'Segmen Risiko',
        'Attrition': 'Status Keluar'
    }
)

# Tambahkan centroid untuk memperjelas pusat cluster
centroids = df_eng.groupby('Cluster')[['Dim1', 'Dim2']].mean().reset_index()
for i, centroid in centroids.iterrows():
    cluster_id = centroid['Cluster']
    label = cluster_map[cluster_id]
    color = color_map[label]
    
    fig.add_trace(
        go.Scatter(
            x=[centroid['Dim1']],
            y=[centroid['Dim2']],
            mode='markers',
            marker=dict(
                symbol='x',
                size=20,
                color=color,
                line=dict(width=2, color='black')
            ),
            name=f'Centroid {label}',
            showlegend=True
        )
    )

fig.update_layout(
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    ),
    height=800,
    width=1000
)
fig.show()

In [ ]:
# 10. Profil karakteristik untuk setiap cluster
# Pilih fitur untuk analisis profil
profile_features = [
    'Age', 'MonthlyIncome', 'DistanceFromHome', 'JobSatisfaction', 
    'JobLevel', 'YearsAtCompany', 'WorkLifeBalance', 'OverTime_Numeric',
    'TotalWorkingYears', 'YearsSinceLastPromotion', 'Satisfaction_Index',
    'Career_Progress', 'Distance_Strain', 'Loyalty_Index'
]

# Hitung statistik per cluster
cluster_profile = df_eng.groupby('Cluster_Label')[profile_features + ['Attrition']].mean()
cluster_profile['Attrition'] *= 100  # Convert to percentage

print("\nProfil karakteristik per cluster:")
print(cluster_profile)

# Visualisasi radar chart untuk profil cluster
# Standarisasi data untuk radar chart
profile_std = (cluster_profile[profile_features] - cluster_profile[profile_features].min()) / (
    cluster_profile[profile_features].max() - cluster_profile[profile_features].min())

# Pilih 8 fitur terpenting untuk radar chart agar tidak terlalu padat
radar_features = [
    'MonthlyIncome', 'OverTime_Numeric', 'JobSatisfaction', 
    'DistanceFromHome', 'YearsAtCompany', 'WorkLifeBalance',
    'Satisfaction_Index', 'Loyalty_Index'
]

# Buat radar chart
fig = go.Figure()

for idx, segment in enumerate(profile_std.index):
    fig.add_trace(go.Scatterpolar(
        r=profile_std.loc[segment, radar_features].values,
        theta=radar_features,
        fill='toself',
        name=f"{segment} (Attrition: {cluster_profile.loc[segment, 'Attrition']:.1f}%)",
        line_color=list(color_map.values())[idx]
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    title='Karakteristik Segmen Karyawan',
    showlegend=True
)

fig.show()


In [ ]:
# 11. Simpan hasil
df_eng.to_csv('employee_segmentation_advanced.csv', index=False)
print(f"\nHasil segmentasi telah disimpan ke 'employee_segmentation_advanced.csv'")

In [ ]:
# 12. Tambahkan ringkasan temuan
print("\n" + "="*50)
print("RINGKASAN SEGMENTASI KARYAWAN")
print("="*50)

# Ringkasan statistik
segment_counts = df_eng['Cluster_Label'].value_counts()
segment_percent = segment_counts / len(df_eng) * 100

print("\nDistribusi Segmen:")
for segment, count in segment_counts.items():
    print(f"{segment}: {count} karyawan ({segment_percent[segment]:.1f}%)")

print("\nTingkat Attrition per Segmen:")
for segment in cluster_profile.index:
    print(f"{segment}: {cluster_profile.loc[segment, 'Attrition']:.1f}%")

print("\nKarakteristik Utama per Segmen:")
for segment in profile_std.index:
    print(f"\n{segment}:")
    
    # Highest values (strengths)
    highest = profile_std.loc[segment].nlargest(3)
    print("  Karakteristik Tertinggi:")
    for feature, value in highest.items():
        original_value = cluster_profile.loc[segment, feature]
        print(f"  - {feature}: {original_value:.2f} (relatif: {value:.2f})")
    
    # Lowest values (weaknesses)
    lowest = profile_std.loc[segment].nsmallest(3)
    print("  Karakteristik Terendah:")
    for feature, value in lowest.items():
        original_value = cluster_profile.loc[segment, feature]
        print(f"  - {feature}: {original_value:.2f} (relatif: {value:.2f})")

print("\nMetode Terbaik:")
print(f"Kombinasi: {best_data_name} + {best_cluster_name}")
print(f"Silhouette Score: {best_silhouette:.4f}")